# Домашнее задание «Продвинутый pandas 2»


**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [13]:
#найдем пользователей, которые поставили более 100 оценок
users = ratings.groupby(ratings['userId']).agg({'rating':'count'})
users = users.loc[users['rating'] > 100]
users.head()

,rating
userId,
4,204
8,116
15,1700
17,363
19,423


In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 4 to 671
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   rating  258 non-null    int64
dtypes: int64(1)
memory usage: 14.0 KB


In [17]:
users.index

Int64Index([  4,   8,  15,  17,  19,  21,  22,  23,  26,  30,
            ...
            647, 648, 652, 654, 655, 656, 659, 664, 665, 671],
           dtype='int64', name='userId', length=258)

In [18]:
# отфильтруем исходный файл по id этих пользователей
filtered_ratings = ratings[ratings['userId'].isin(users.index)]
filtered_ratings.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [22]:
# найдем минимум и максимум показателя timestamp 
filtered_ratings = filtered_ratings.groupby('userId').agg({'timestamp':['min', 'max']})
filtered_ratings

timestamp            
               min         max
userId                        
4        949778714   949982274
8       1154389340  1154474527
15       997937239  1469330735
17      1127468587  1127476640
19       855190091   855195373
...            ...         ...
656      986240991   986244044
659      834598040   866207451
664     1343731283  1441911722
665      992836298  1046967769
671     1063500751  1074784735

[258 rows x 2 columns]

In [23]:
# вычислим время жизни пользователей
filtered_ratings.loc[: , 'diff'] = filtered_ratings['timestamp']['max'] - filtered_ratings['timestamp']['min']
filtered_ratings.head()

timestamp                   diff
               min         max           
userId                                   
4        949778714   949982274     203560
8       1154389340  1154474527      85187
15       997937239  1469330735  471393496
17      1127468587  1127476640       8053
19       855190091   855195373       5282

In [26]:
# это среднее время жизни в секундах
filtered_ratings['diff'].mean()

40080507.4496124

In [41]:
# переведем значение в дни для наглядности
# получаем среднее время жизни пользователя почти полтора года
round(filtered_ratings['diff'].mean() / (60 * 60 * 24), 2)

463.89

**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [42]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [43]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [44]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [45]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [53]:
merged_1 = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
merged_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [54]:
merged_2 = client_base.merge(rzd,on='client_id', how='left').merge(auto,on='client_id', how='left').merge(air,on='client_id', how='left')
merged_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

- Если нам предоставлены отдельно таблица визитов/покупок и набор координат, возможно, получится связать их воедино, если есть еще и временные отметки в обеих таблицах. Т.е. если пользователь в определенное время находился в точке с координатами, которые мы знаем, и в это же время им сделана покупка, то мы можем привязать эту пару координат к идентификатору покупки. 

- В целом, кажется логичным, чтобы приложение во время работы привязывало пару координат к какому-то конкретному действию пользователя, если нам нужны геоданные. Переход на сайт, совершение покупки - все это дискретные действия, на них можно навесить идентификатор и к нему прицеплять пару координат. И в итоге получать таблицу вида:



In [2]:
df = pd.DataFrame({
    'visit_id':['001', '002', '003'],
    'latitude':[42.5666667, 42.4833333, 42.6166667],
    'longitude':[1.5166667, 1.4666667, 1.6333333]
})
df

,visit_id,latitude,longitude
0,001,42.566667,1.516667
1,002,42.483333,1.466667
2,003,42.616667,1.633333


По поводу практического использования. Во-первых, можно попробовать подобрать к паре координат соответствующий населенный пункт, тогда это будет уже не просто пара чисел. В открытом доступе есть датасеты, содержащие информацию о городах с их координатами широты и долготы, например, такой: https://www.kaggle.com/max-mind/world-cities-database



Беглый гуглопоиск показал, что и с помощью библиотек Питона типа Basemap можно наносить точки именно по координатам на реальную карту мира. Т.е. в принципе, можно построить визуальную модель распределения и плотности пользователей на карте. 

В целом, знание этих данных может как-то помочь скорректировать процессы логистики, если речь идет, допустим, об интернет-магазине. Если видно, что в каком-то городе достаточно много пользователей совершает покупки, то имеет смысл разместить там свои склады и пункты выдачи, чтобы сократить время доставки. Или, зная, что в одном регионе повышен спрос на определенную группу товаров, а в другом понижен, скорректировать объемы поставок в соответствии со спросом.